# Acuity Classification Comparison: GPT-3.5 vs GPT-4o
This notebook runs the acuity classification task using both GPT-3.5 and GPT-4o, allowing side-by-side comparison of model predictions.

In [ ]:
import pandas as pd
import openai
import time

# Load and inspect dataset
df = pd.read_csv('results.csv')
df.head()

In [ ]:
# Define the prompt template based on original notebooks
def format_prompt(row):
    prompt = "Predict the emergency department acuity level for this patient.\n\n"
    if 'primarychiefcomplaintname' in row:
        prompt += f"Chief Complaint: {row['primarychiefcomplaintname']}\n"
    if 'age' in row:
        prompt += f"Age: {row['age']}\n"
    if 'sex' in row:
        prompt += f"Sex: {row['sex']}\n"
    if 'firstrace' in row:
        prompt += f"Race: {row['firstrace']}\n"
    if 'Vital_Signs' in row:
        prompt += f"Vital Signs: {row['Vital_Signs']}\n"
    prompt += "\nSelect the most appropriate acuity level from the following options ONLY:\n"
    prompt += "'Immediate', 'Emergent', 'Urgent', 'Less Urgent', 'Non-Urgent'\n\n"
    prompt += "Respond with ONLY ONE of these five options."
    return prompt

In [ ]:
# Query the OpenAI API
def get_model_response(prompt, model_name='gpt-3.5-turbo'):
    try:
        response = openai.ChatCompletion.create(
            model=model_name,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1
        )
        return response['choices'][0]['message']['content']
    except Exception as e:
        print(f"Error: {e}")
        return None

In [ ]:
# Run predictions for both models
results = []
for i, row in df.iterrows():
    prompt = format_prompt(row)
    out_35 = get_model_response(prompt, 'gpt-3.5-turbo')
    time.sleep(1)
    out_4o = get_model_response(prompt, 'gpt-4o')
    results.append({
        'index': i,
        'prompt': prompt,
        'gpt-3.5-turbo': out_35,
        'gpt-4o': out_4o,
        'gold': row.get('acuitylevel', None)
    })

In [ ]:
# Convert results to DataFrame and display
results_df = pd.DataFrame(results)
results_df.to_csv('acuity_comparison_results.csv', index=False)
results_df[['gpt-3.5-turbo', 'gpt-4o', 'gold']].head()